# FAQ recommendations

This Jupyter Notebook conducts an evaluation of various approaches to develop an LLM algorithm capable of recommending solutions based on an F.A.Q. database and the conversation context between an agent and a client.

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# 1 handling the data

## 1.1.0 Converting F.A.Q from .txt to .csv file

In [30]:
def get_questions_and_answers(faq_path: str):
    questions = []
    answers = []

    with open(faq_path, "r") as file:
        for line in file.readlines():
            if "Pergunta" in line:
                question = line.lstrip("Pergunta: ").rstrip("\n")
                questions.append(question)
            elif "Resposta" in line:
                answer = line.lstrip("Resposta: ").rstrip("\n")
                answers.append(answer)
    
    return (questions, answers)


def create_csv(questions: list, answers: list, destination_path: str, file_name: str):
    faq_dict = {
        "Question": questions,
        "Answer": answers
        }
    
    path = destination_path + file_name + ".csv"

    df = pd.DataFrame(data=faq_dict, dtype=object)
    df.to_csv(path_or_buf=path, sep=";", index=False)

In [31]:
questions, answers = get_questions_and_answers(faq_path="data/raw/FAQ_example.txt")
create_csv(questions=questions, answers=answers, destination_path="data/raw/", file_name="FAQ_example")

# 2 Testing LLMs techniques

## 2.1 Sentence Embeddings pretrained models

In [ ]:
%%python -m spacy download pt_core_news_sm

from sentence_transformers import SentenceTransformer, util
import spacy
import re


def get_greatest_similarities(model, dataframe, message):
    questions_embeddings = model.encode(dataframe.Question)
    message_embed = model.encode(get_clean_text(message))

    similarities = util.cos_sim(message_embed, questions_embeddings)[0].tolist()
    greatest_similarities_index = pd.Series(similarities).sort_values(ascending=False).index
    
    return greatest_similarities_index


def get_clean_text(text:str):
    nlp_model = spacy.load("pt_core_news_sm")

    text_tokens = nlp_model(text)

    clean_text = []

    for token in text_tokens:
        if not token.is_stop and not token.is_punct:
            clean_text.append(token.text)

    clean_text = " ".join(clean_text)

    return re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', clean_text).lower()

In [33]:
df = pd.read_csv("data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


In [34]:
message = "Como que eu faço para contratar um serviço melhor?"
message = get_clean_text(message)
print(message)

contratar serviço melhor


### 2.1.1 all-mpnet-base-v2 model

In [35]:
model = SentenceTransformer("all-mpnet-base-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."


### 2.1.2 all-MiniLM-L6-v2 model

In [36]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."


### 2.1.3 msmarco-distilbert-base-v4

In [37]:
model = SentenceTransformer("msmarco-distilbert-base-v4")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
21,A empresa disponibiliza serviços de telefonia ...,"Sim, oferecemos opções para ligações internaci..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...


### 2.1.4 bert-large-portuguese-cased (Finetuned model)

In [38]:
from sentence_transformers import models

model = models.Transformer("models/bert-large-portuguese-cased")
pooling_model = models.Pooling(model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[model, pooling_model])

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
21,A empresa disponibiliza serviços de telefonia ...,"Sim, oferecemos opções para ligações internaci..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
29,A empresa disponibiliza pacotes de internet co...,"Sim, temos opções de planos com tráfego ilimit..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...


# 3 Preparing to Deploy

In [49]:
from sentence_transformers import models

word_embedding_model = models.Transformer("models/bert-large-portuguese-cased")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

embedding_model = SentenceTransformer(modules=[model, pooling_model])

In [50]:
nlp_model = spacy.load("pt_core_news_sm")

In [51]:
df = pd.read_csv("data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


## 3.1 Preparing Data

### 3.1.1 Creating Embeddings for each question and exporting as CSV file

In [52]:
def get_sentence_embeddings(model, series):
    """This function removes all 'stop words' for each sentence in a pandas.Series
    and then creates a embed for the sentence without the stop words."""

    for i, text in enumerate(series):
        embed = model.encode(get_clean_text(text))
        
        if i == 0:
            text_embeddings = pd.DataFrame({f"{i}": embed})
        else:
            text_embeddings[f"{i}"] = embed

    return text_embeddings

In [53]:
df_embedding = get_sentence_embeddings(embedding_model, df.Question)
df_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.188625,0.400796,0.411759,0.109735,0.009841,0.034627,0.166421,0.056301,0.097398,-0.092919,...,-0.413870,-0.321135,0.878566,-0.406815,-0.006662,0.082112,-0.196085,-0.239688,0.049380,-0.480112
1,0.669775,-0.513453,0.729795,0.126028,0.267572,0.481285,1.141499,0.142642,0.742798,0.549269,...,0.589086,0.127874,0.220501,0.422828,-0.126927,0.728832,0.353417,0.152385,-0.145056,0.394541
2,-0.381557,-0.054400,-0.294071,0.117846,-0.608610,-0.065050,-0.305382,0.236780,-0.003630,0.204660,...,0.205726,-0.050292,0.137386,-0.305941,-0.038119,-0.013325,-0.102130,-0.320344,0.274796,-0.026570
3,0.454016,0.232534,0.188410,-0.255084,0.524474,0.415265,-0.067251,0.152116,0.100762,0.834601,...,-0.138930,0.220052,0.151823,0.536145,0.055320,0.060069,0.320545,0.123530,-0.049229,0.277208
4,0.066648,-0.130414,0.453748,-0.097489,-0.049003,-0.478443,0.460942,0.129103,0.424205,-0.115655,...,0.023551,0.136368,0.457364,0.372163,0.002983,0.833863,0.270272,-0.148331,0.182577,0.618017


In [56]:
df_embedding.to_csv(path_or_buf="data/processed/Embeddings.csv", sep=";", index=False)

## 3.2 Exporting the model with the best performance

### 3.2.1 Embeddings model

In [58]:
embedding_model.save('models/embeddings_model')

### 3.2.2 NLP Model

In [59]:
nlp_model.to_disk('models/nlp_model')